In [27]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [28]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [29]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir,label)):
            image_paths.append(os.path.join(dir,label,imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths,labels


In [30]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe("../DATASETS/train")

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [31]:
print(train)

                                                  image     label
0         ../DATASETS/train\angry\Training_10118481.jpg     angry
1         ../DATASETS/train\angry\Training_10120469.jpg     angry
2         ../DATASETS/train\angry\Training_10131352.jpg     angry
3         ../DATASETS/train\angry\Training_10161559.jpg     angry
4          ../DATASETS/train\angry\Training_1021836.jpg     angry
...                                                 ...       ...
28704  ../DATASETS/train\surprise\Training_99916297.jpg  surprise
28705  ../DATASETS/train\surprise\Training_99924420.jpg  surprise
28706  ../DATASETS/train\surprise\Training_99937001.jpg  surprise
28707  ../DATASETS/train\surprise\Training_99951755.jpg  surprise
28708  ../DATASETS/train\surprise\Training_99984132.jpg  surprise

[28709 rows x 2 columns]


In [32]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe("../DATASETS/test")

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed


In [33]:
print(test)
print(test['image'])

                                                  image     label
0       ../DATASETS/test\angry\PrivateTest_10131363.jpg     angry
1       ../DATASETS/test\angry\PrivateTest_10304478.jpg     angry
2        ../DATASETS/test\angry\PrivateTest_1054527.jpg     angry
3       ../DATASETS/test\angry\PrivateTest_10590091.jpg     angry
4        ../DATASETS/test\angry\PrivateTest_1109992.jpg     angry
...                                                 ...       ...
7173  ../DATASETS/test\surprise\PublicTest_98089595.jpg  surprise
7174  ../DATASETS/test\surprise\PublicTest_98567249.jpg  surprise
7175  ../DATASETS/test\surprise\PublicTest_98972870.jpg  surprise
7176  ../DATASETS/test\surprise\PublicTest_99242645.jpg  surprise
7177  ../DATASETS/test\surprise\PublicTest_99446963.jpg  surprise

[7178 rows x 2 columns]
0         ../DATASETS/test\angry\PrivateTest_10131363.jpg
1         ../DATASETS/test\angry\PrivateTest_10304478.jpg
2          ../DATASETS/test\angry\PrivateTest_1054527.jpg
3        

In [34]:
from tqdm.notebook import tqdm

In [35]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image,grayscale =  True )
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features),48,48,1)
    return features
    

In [36]:
train_features = extract_features(train['image']) 

  0%|          | 0/28709 [00:00<?, ?it/s]

c:\Users\shiva\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


In [37]:
test_features = extract_features(test['image'])

  0%|          | 0/7178 [00:00<?, ?it/s]

In [38]:
x_train = train_features/255.0
x_test = test_features/255.0

In [39]:
from sklearn.preprocessing import LabelEncoder

In [40]:
le = LabelEncoder()
le.fit(train['label'])

LabelEncoder()

In [41]:
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])

In [42]:
y_train = to_categorical(y_train,num_classes = 7)
y_test = to_categorical(y_test,num_classes = 7)

In [43]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(7, activation='softmax'))

In [44]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [45]:
model.fit(x= x_train,y = y_train, batch_size = 128, epochs = 1, validation_data = (x_test,y_test)) 

225/225 [==============================] - 550s 2s/step - loss: 1.8233 - accuracy: 0.2459 - val_loss: 1.8115 - val_accuracy: 0.2471


In [46]:
model_json = model.to_json()
with open("emotiondetector.json",'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")

In [47]:
from keras.models import model_from_json

In [48]:
json_file = open("facialemotionmodel.json", "r")
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights("facialemotionmodel.h5")

In [49]:
label = ['angry','disgust','fear','happy','neutral','sad','surprise']

In [50]:
def ef(image):
    img = load_img(image,grayscale =  True )
    feature = np.array(img)
    feature = feature.reshape(1,48,48,1)
    return feature/255.0
    

In [51]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)

original image is of sad


c:\Users\shiva\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


FileNotFoundError: [Errno 2] No such file or directory: 'images/train/sad/42.jpg'

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

: 

In [ ]:
image = 'images/train/sad/42.jpg'
print("original image is of sad")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

: 

In [ ]:
image = 'images/train/fear/2.jpg'
print("original image is of fear")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

: 

In [ ]:
image = 'images/train/disgust/299.jpg'
print("original image is of disgust")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

: 

In [ ]:
image = 'images/train/happy/7.jpg'
print("original image is of happy")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

: 

In [ ]:
image = 'images/train/surprise/15.jpg'
print("original image is of surprise")
img = ef(image)
pred = model.predict(img)
pred_label = label[pred.argmax()]
print("model prediction is ",pred_label)
plt.imshow(img.reshape(48,48),cmap='gray')

: 

: 